In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flip-or-not-flip/images/training/notflip/0036_000000026.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0044_000000017.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0004_000000024.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0035_000000029.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0041_000000005.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0032_000000024.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0005_000000004.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0045_000000001.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0026_000000017.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0057_000000025.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0035_000000031.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0033_000000013.jpg
/kaggle/input/flip-or-not-flip/images/training/notflip/0058_000000023.jpg
/kaggle/input/flip-or-not-flip/images/

In [12]:
originaldata_dir = '../input/flip-or-not-flip/images'
train_dir = originaldata_dir + '/training'
test_dir = originaldata_dir + '/testing'

# Split train data and validation data

In [13]:

import os
import shutil
from sklearn.model_selection import train_test_split

# 创建文件夹
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)
        print(f'-- new folder "{path}" --')
    else:
        print(f'-- the folder "{path}" is already here --')
# read the flip and notflip folds of original training data
dataset_path_flip = "../input/flip-or-not-flip/images/training/flip"
dataset_path_notflip = "../input/flip-or-not-flip/images/training/notflip"

# create the  save path of flip and notflip folds on training and validation data
train_set_save_path_flip = "./training/flip"
train_set_save_path_notflip = "./training/notflip"
val_set_save_path_flip = "./validation/flip"
val_set_save_path_notflip = "./validation/notflip"

# save the path 
mkdir(train_set_save_path_flip)
mkdir(train_set_save_path_notflip)
mkdir(val_set_save_path_flip)
mkdir(val_set_save_path_notflip)

#read the original flip and notflip folds of original training data 

file_pathes_flip = os.listdir(dataset_path_flip)
file_pathes_notflip = os.listdir(dataset_path_notflip)
# print(file_pathes_flip)

# get training fold all the flip image name 
img_names_flip = []
for file_path in file_pathes_flip:
    if os.path.splitext(file_path)[1] == ".jpg":
        file_name = os.path.splitext(file_path)[0]
        img_names_flip.append(file_name)


# getting trainning fold all notfilp image name 
img_names_notflip = []
for file_path in file_pathes_notflip:
    if os.path.splitext(file_path)[1] == ".jpg":
        file_name = os.path.splitext(file_path)[0]
        img_names_notflip.append(file_name)


        
        
# split the test and validation data from flip and notflip folds
train_set_flip, val_set_flip = train_test_split(img_names_flip, test_size=0.2, random_state=42)
train_set_notflip, val_set_notflip = train_test_split(img_names_notflip, test_size=0.2, random_state=42)

# print(f"train_set size: {len(train_set)}, val_set size: {len(val_set)}")


# train solve: move train_set_flip to target fold
for file_name in train_set_flip:
    train_img_src_path_flip = os.path.join(dataset_path_flip, file_name+".jpg")
    train_img_dst_path_flip = os.path.join(train_set_save_path_flip, file_name+".jpg")
    shutil.copyfile(train_img_src_path_flip,train_img_dst_path_flip)


# train solve: move train_set_notflip to target fold
for file_name in train_set_notflip:
    train_img_src_path_notflip = os.path.join(dataset_path_notflip, file_name+".jpg")
    train_img_dst_path_notflip = os.path.join(train_set_save_path_notflip, file_name+".jpg")
    shutil.copyfile(train_img_src_path_notflip,train_img_dst_path_notflip)
    
# validation solve: move val_set_flip to target fold
for file_name in val_set_flip:
    val_img_src_path_flip = os.path.join(dataset_path_flip, file_name+".jpg")
    val_img_dst_path_flip = os.path.join(val_set_save_path_flip, file_name+".jpg")
    shutil.copyfile(val_img_src_path_flip,val_img_dst_path_flip)   

# validation solve: move val_set_flip to target fold
for file_name in val_set_notflip:
    val_img_src_path_notflip = os.path.join(dataset_path_notflip, file_name+".jpg")
    val_img_dst_path_notflip = os.path.join(val_set_save_path_notflip, file_name+".jpg")
    shutil.copyfile(val_img_src_path_notflip,val_img_dst_path_notflip)   
 


-- the folder "./training/flip" is already here --
-- the folder "./training/notflip" is already here --
-- the folder "./validation/flip" is already here --
-- the folder "./validation/notflip" is already here --


# Set model

In [14]:
from torch import nn
import torch
def _make_divisible(ch, divisor=8, min_ch=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_ch is None:
        min_ch = divisor
    new_ch = max(min_ch, int(ch + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_ch < 0.9 * ch:
        new_ch += divisor
    return new_ch


class ConvBNReLU(nn.Sequential):
    def __init__(self, in_channel, out_channel, kernel_size=3, stride=1, groups=1):
        padding = (kernel_size - 1) // 2
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding, groups=groups, bias=False),
            nn.BatchNorm2d(out_channel),
            nn.ReLU6(inplace=True)
        )


class InvertedResidual(nn.Module):
    def __init__(self, in_channel, out_channel, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        hidden_channel = in_channel * expand_ratio
        self.use_shortcut = stride == 1 and in_channel == out_channel

        layers = []
        if expand_ratio != 1:
            # 1x1 pointwise conv
            layers.append(ConvBNReLU(in_channel, hidden_channel, kernel_size=1))
        layers.extend([
            # 3x3 depthwise conv
            ConvBNReLU(hidden_channel, hidden_channel, stride=stride, groups=hidden_channel),
            # 1x1 pointwise conv(linear)
            nn.Conv2d(hidden_channel, out_channel, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channel),
        ])

        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_shortcut:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000, alpha=1.0, round_nearest=8):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = _make_divisible(32 * alpha, round_nearest)
        last_channel = _make_divisible(1280 * alpha, round_nearest)

        inverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        features = []
        # conv1 layer
        features.append(ConvBNReLU(3, input_channel, stride=2))
        # building inverted residual residual blockes
        for t, c, n, s in inverted_residual_setting:
            output_channel = _make_divisible(c * alpha, round_nearest)
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(block(input_channel, output_channel, stride, expand_ratio=t))
                input_channel = output_channel
        # building last several layers
        features.append(ConvBNReLU(input_channel, last_channel, 1))
        # combine feature layers
        self.features = nn.Sequential(*features)

        # building classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(last_channel, num_classes)
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

### I choose the mobilenetv2 as train model because this model is the smallest trained models with comparatively less number of convolution layers. It’s faster to train on images similarly to image net dataset.


In [15]:
import os
import sys
import json

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from tqdm import tqdm

In [16]:
data_dir = './'
train_dir = data_dir + '/training'
validation_dir = data_dir + '/validation'

# Train model

In [17]:



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

batch_size = 16
epochs = 10

data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
        "validation": transforms.Compose([transforms.Resize(256),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

train_dataset = datasets.ImageFolder(root=os.path.join(train_dir),
                                         transform=data_transform["train"])
train_num = len(train_dataset)

nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
print('Using {} dataloader workers every process'.format(nw))

train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size, shuffle=True,
                                               num_workers=nw)

validation_dataset = datasets.ImageFolder(root=os.path.join(validation_dir),
                                            transform=data_transform["validation"])
validation_num = len(validation_dataset)
validation_loader = torch.utils.data.DataLoader(validation_dataset,
                                                  batch_size=batch_size, shuffle=False,
                                                  num_workers=nw)

print("using {} images for training, {} images for validation.".format(train_num,
                                                                           validation_num))

# create model
net = MobileNetV2(num_classes=2)

    
# unfreeze features weights
for param in net.features.parameters():
    param.requires_grad = True  

net.to(device)

# define loss function
loss_function = nn.CrossEntropyLoss()

# construct an optimizer
params = [p for p in net.parameters() if p.requires_grad]
optimizer = optim.Adam(params, lr=0.0001)

best_acc = 0.0
save_path = './MobileNetV2.pth'
train_steps = len(train_loader)
for epoch in range(epochs):
    # train
    net.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, file=sys.stdout)
    for step, data in enumerate(train_bar):
        images, labels = data
        optimizer.zero_grad()
        logits = net(images.to(device))
        loss = loss_function(logits, labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)

    # validation
    net.eval()
    acc = 0.0  # accumulate accurate number / epoch
    with torch.no_grad():
        validation_bar = tqdm(validation_loader, file=sys.stdout)
        for validation_data in validation_bar:
            validation_images, validation_labels = validation_data
            outputs = net(validation_images.to(device))
            # loss = loss_function(outputs, test_labels)
            predict_y = torch.max(outputs, dim=1)[1]
            acc += torch.eq(predict_y, validation_labels.to(device)).sum().item()

            validation_bar.desc = "test epoch[{}/{}]".format(epoch + 1,
                                                           epochs)
    validation_accurate = acc / validation_num
    print('[epoch %d] train_loss: %.3f  validation_accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, validation_accurate))

    if validation_accurate > best_acc:
        best_acc = validation_accurate
        torch.save(net.state_dict(), save_path)

print('Finished Training')
print(predict_y)




using cpu device.
Using 4 dataloader workers every process
using 1913 images for training, 479 images for validation.
test epoch[1/10]: 100%|██████████| 30/30 [00:13<00:00,  2.22it/s]
[epoch 1] train_loss: 0.689  validation_accuracy: 0.628
test epoch[2/10]: 100%|██████████| 30/30 [00:13<00:00,  2.19it/s]
[epoch 2] train_loss: 0.587  validation_accuracy: 0.597
test epoch[3/10]: 100%|██████████| 30/30 [00:13<00:00,  2.24it/s]
[epoch 3] train_loss: 0.516  validation_accuracy: 0.770
test epoch[4/10]: 100%|██████████| 30/30 [00:13<00:00,  2.23it/s]
[epoch 4] train_loss: 0.509  validation_accuracy: 0.827
test epoch[5/10]: 100%|██████████| 30/30 [00:13<00:00,  2.21it/s]
[epoch 5] train_loss: 0.483  validation_accuracy: 0.837
test epoch[6/10]: 100%|██████████| 30/30 [00:13<00:00,  2.23it/s]
[epoch 6] train_loss: 0.464  validation_accuracy: 0.858
test epoch[7/10]: 100%|██████████| 30/30 [00:13<00:00,  2.22it/s]
[epoch 7] train_loss: 0.455  validation_accuracy: 0.843
test epoch[8/10]: 100%|█████

### Apply Transformer technical to pretrained mobilenetv2 model,e.g.RandomResizedCrop,RandomHorizontalFlip, and Normalize for training step, CenterCrop, and Normalize fro validation step. 
### Unfreeze the feature weights for getting the image feature in our projects and train the dense layer for classification for training step and validation step using PyTorch framework for 10 epoch. 

In [18]:
originaldata_dir = '../input/flip-or-not-flip/images'
test_dir = originaldata_dir + '/testing'

In [19]:
"""
data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
        "test": transforms.Compose([transforms.Resize(256),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

test_dataset = datasets.ImageFolder(root=os.path.join(test_dir),
                                            transform=data_transform["test"])

test_loader = torch.utils.data.DataLoader(test_dataset,
                                                  batch_size=batch_size, shuffle=False,
                                                  num_workers=nw)


with torch.no_grad():
        test_bar = tqdm(test_loader, file=sys.stdout)
        for test_data in test_bar:
            test_images, test_labels = test_data
            outputs = net(test_images.to(device))
            # loss = loss_function(outputs, test_labels)
            predict_y = torch.max(outputs, dim=1)[1]
"""

'\ndata_transform = {\n        "train": transforms.Compose([transforms.RandomResizedCrop(224),\n                                     transforms.RandomHorizontalFlip(),\n                                     transforms.ToTensor(),\n                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),\n        "test": transforms.Compose([transforms.Resize(256),\n                                   transforms.CenterCrop(224),\n                                   transforms.ToTensor(),\n                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}\n\ntest_dataset = datasets.ImageFolder(root=os.path.join(test_dir),\n                                            transform=data_transform["test"])\n\ntest_loader = torch.utils.data.DataLoader(test_dataset,\n                                                  batch_size=batch_size, shuffle=False,\n                                                  num_workers=nw)\n\n\nwith t

In [20]:
batch_size = 16
epochs = 10

### set the batch_size and epochs for testing step

# Test Model

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_transform = {
        
        "test": transforms.Compose([transforms.Resize(256),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}
batch_size = 16
epochs = 10
                                
test_dataset = datasets.ImageFolder(root=os.path.join(test_dir),
                                            transform=data_transform["test"])
test_num = len(test_dataset)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                                  batch_size=batch_size, shuffle=False,
                                          num_workers=nw)

# create model

net.eval()
target_num = torch.zeros((1, 2)) # n_classes为分类任务类别数量
predict_num = torch.zeros((1, 2))
acc_num = torch.zeros((1, 2))
with torch.no_grad():
    test_bar = tqdm(test_loader, file=sys.stdout)
    for test_data in test_bar:
        test_images, test_labels = test_data
        outputs = net(test_images.to(device))
        # loss = loss_function(outputs, test_labels)
        predict_y = torch.max(outputs, dim=1)[1]
        pre_mask = torch.zeros(outputs.size()).scatter_(1, predict_y.cpu().view(-1, 1), 1.)
        predict_num += pre_mask.sum(0)  # get the number of each class predict 
        print(predict_num)
        tar_mask = torch.zeros(outputs.size()).scatter_(1, test_labels.data.cpu().view(-1, 1), 1.)
        target_num += tar_mask.sum(0)  
        acc_mask = pre_mask * tar_mask 
        acc_num += acc_mask.sum(0) #get the number of the accurate classification predicted 
    recall = acc_num / target_num
    precision = acc_num / predict_num
    F1 = 2 * recall * precision / (recall + precision)
    
    print('Test  F1-score {}'.format(F1))

100%|██████████| 38/38 [00:16<00:00,  2.24it/s]
Test  F1-score tensor([[0.8608, 0.8799]])


### This step is for testing. set 10 epochs for testing and get F1 score of 0.8799, not too bad. 